In [3]:
import pandas as pd
from astroquery.ipac.irsa import Irsa
from astropy.coordinates import SkyCoord
import astropy.units as u

In [11]:
# Cargar el catálogo SPLSU
splus_data = pd.read_csv('../Ha-emitters/Halpha_Mine_PerField_total-unique.csv')  # Ajusta el nombre del archivo

In [28]:
# Ensure RA and DEC are in the correct units, converting them to quantities with degrees
splus_coords = SkyCoord(ra=splus_data['RA'] * u.deg, dec=splus_data['DEC'] * u.deg, frame='icrs')

UnitTypeError: Angle instances require units equivalent to 'rad', but no unit was given.

In [21]:
# Now, explicitly create the SkyCoord object
try:
    splus_coords = SkyCoord(ra=splus_data['RA'] * u.deg, dec=splus_data['DEC'] * u.deg)
    print(splus_coords)
except Exception as e:
    print(f"Error: {e}")

Error: Angle instances require units equivalent to 'rad', but no unit was given.


In [8]:
# Definir el radio de búsqueda (por ejemplo, 0.5 grados)
radius = 0.5  # En grados

In [9]:
# Realizar el cross-match entre SPLSU y WISE
for i, splsu_row in splsu_data.iterrows():
    # Extraer las coordenadas RA y DEC de SPLSU
    ra_splsu = splsu_row['RA']
    dec_splsu = splsu_row['DEC']
    
    # Consultar el catálogo WISE para la fuente actual de SPLUS
    wise_results = query_wise(ra_splsu, dec_splsu, radius)
    
    # Si hay resultados, agregar la mejor coincidencia (puedes aplicar más filtros si es necesario)
    if len(wise_results) > 0:
        best_match = wise_results.iloc[0]  # Tomamos la primera coincidencia (la más cercana)
        
        # Almacenar la coincidencia en la lista
        matches.append({
            'SPLSU_RA': ra_splsu,
            'SPLSU_DEC': dec_splsu,
            'WISE_RA': best_match['RA'],
            'WISE_DEC': best_match['DEC'],
            'WISE_Object_ID': best_match['designation'],
            'WISE_Flux': best_match['w1mpro'],  # Asumiendo que 'w1mpro' es el flujo en la banda W1
            'WISE_Flux_Error': best_match['w1sigmpro'],  # Error en el flujo
            'Distance_deg': SkyCoord(ra=ra_splsu, dec=dec_splsu, unit=(u.deg, u.deg)).separation(SkyCoord(ra=best_match['RA'], dec=best_match['DEC'], unit=(u.deg, u.deg))).deg
        })

DALQueryError: UsageFault: BAD_REQUEST: unknown table: wise